In [1]:
import yfinance as yf
import numpy as np
import datetime as dt

In [3]:
ticker = '^GSPC'

In [4]:
start = dt.date.today() - dt.timedelta(1825)
end = dt.date.today()

In [5]:
SnP = yf.download(ticker, start, end)

[*********************100%***********************]  1 of 1 downloaded


In [6]:
SnP.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-09-14,1963.06,1963.06,1948.27,1953.03,1953.03,3000200000
2015-09-15,1955.10,1983.19,1954.30,1978.09,1978.09,3239860000
2015-09-16,1978.02,1997.26,1977.93,1995.31,1995.31,3630680000
2015-09-17,1995.33,2020.86,1986.73,1990.20,1990.20,4183790000
2015-09-18,1989.66,1989.66,1953.45,1958.03,1958.03,6021240000


In [9]:
def CAGR(DF):
    """
    Function to calculate the Cumulative Annual Growth Rate of a given data table
    
    """
    df = DF.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    df['cum_ret'] = (1 + df['daily_ret']).cumprod()
    n = len(df)/252
    CAGR = (df['cum_ret'][-1])**(1/n) - 1
    return CAGR    

In [14]:
CAGR(SnP)

0.11748651335385318

In [15]:
def volatility(DF):
    "function to calculate annualized volatility of a trading strategy"
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    vol = df["daily_ret"].std() * np.sqrt(252)
    return vol

In [16]:
volatility(SnP)

0.19170333193252134

Measuring Sharpe and Sortino Ratios

In [17]:
def sharpe(DF,rf):
    "function to calculate sharpe ratio ; rf is the risk free rate"
    df = DF.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr

In [19]:
sharpe(SnP, 0.00)

0.6128558756360473

In [20]:
def sortino(DF,rf):
    "function to calculate sortino ratio ; rf is the risk free rate"
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    df["neg_ret"] = np.where(df["daily_ret"]<0,df["daily_ret"],0)
    neg_vol = df["neg_ret"].std() * np.sqrt(252)
    sr = (CAGR(df) - rf)/neg_vol
    return sr

In [21]:
sortino(SnP, 0)

0.9076588474816759

Maximum Drawdown & Calmar Ratio

In [22]:
def max_dd(DF):
    "function to calculate max drawdown"
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    df["cum_return"] = (1 + df["daily_ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

In [23]:
max_dd(SnP)

0.33924959024260576

In [24]:
def calmar(DF):
    "function to calculate calmar ratio"
    df = DF.copy()
    clmr = CAGR(df)/max_dd(df)
    return clmr

In [25]:
calmar(SnP)

0.34631291159360184